# Generate Answer based on T5 Summary 

In [2]:
from datasets import load_dataset 
dataset = load_dataset("khalidrizki/RECOMP-tuning-truncated")
test_data = dataset["test"]

c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since khalidrizki/RECOMP-tuning-truncated couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\LENOVO\.cache\huggingface\datasets\khalidrizki___recomp-tuning-truncated\default\0.0.0\0af0dd211ef16e1ecae2ee0d4b857744a1ad1e05 (last modified on Fri Jun 13 11:24:20 2025).


In [4]:
from utils import load_model_and_tokenizer
model_name='Qwen/Qwen3-1.7B'
model, tokenizer, config = load_model_and_tokenizer(model_name)

Loading model on cuda with torch.bfloat16


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.37it/s]


## Dataset terbaru

### Selective

In [ ]:
# Muat baris-baris teks dari file
with open("./outputs/FINAL-seeded-truncated-khalidrizki-RECOMP-selective-final-2025-06-13_11-23-00/generated_predictions.txt", "r", encoding="utf-8", errors='replace') as f:
    predictions = [line.strip() for line in f.readlines()]

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions) == len(test_data), f"Jumlah prediksi ({len(predictions)}) tidak sama dengan jumlah data ({len(test_data)})"

In [ ]:
# Tambahkan kolom ke dataset
test_data = test_data.add_column("T5_summary", predictions)

from generate_answer import generate_answer_and_do_scoring

answer_based_on_t5_summary = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    filtered_contexts_col="T5_summary", 
    label_col='answer', 
    original_passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52
)

Generating responses with summary only:   0%|          | 0/565 [00:00<?, ?it/s]c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset

In [6]:
from datasets import Dataset
result = Dataset.from_list(answer_based_on_t5_summary)
em_avg = sum(result['em'])/len(result)
f1_avg =  sum(result['f1'])/len(result)
print("EM:" ,em_avg)
print("F1:", f1_avg)

EM: 0.3663716814159292
F1: 0.2368892156786773


### Unselective

In [ ]:
# Muat baris-baris teks dari file
with open("./outputs/FINAL-seeded-truncated-khalidrizki-RECOMP-unselective-final-2025-06-13_10-51-32/generated_predictions.txt", "r", encoding="utf-8", errors='replace') as f:
    predictions_unselective = [line.strip() for line in f.readlines()]

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions_unselective) == len(test_data), f"Jumlah prediksi ({len(predictions_unselective)}) tidak sama dengan jumlah data ({len(test_data)})"

# Tambahkan kolom ke dataset
test_data = test_data.add_column("unselective_summary", predictions_unselective)

from generate_answer import generate_answer_and_do_scoring

answer_based_on_unselective_summary = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    filtered_contexts_col="unselective_summary", 
    label_col='answer', 
    original_passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52
)

from datasets import Dataset
result_uns = Dataset.from_list(answer_based_on_unselective_summary)
em_uns_avg = sum(result_uns['em'])/len(result_uns)
f1_uns_avg =  sum(result_uns['f1'])/len(result_uns)
print("EM:" ,em_uns_avg)
print("F1:", f1_uns_avg)

Generating responses with summary only: 100%|██████████| 565/565 [24:46<00:00,  2.63s/it]

EM: 0.4230088495575221
F1: 0.25677911467294434


### RAG Konvensional

In [ ]:
from generate_answer import generate_answer_and_do_scoring
answer_based_on_passages = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    filtered_contexts_col="passages", 
    label_col='answer', 
    original_passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52, 
    max_source_length = 512
)

from datasets import Dataset
normal_RAG = Dataset.from_list(answer_based_on_passages)
print("HASIL RAG BIASA")
print("rerata EM:", sum(normal_RAG['em'])/len(normal_RAG))
print("rerata F1:", sum(normal_RAG['f1'])/len(normal_RAG))

Generating responses with summary only: 100%|██████████| 565/565 [28:18<00:00,  3.01s/it]

HASIL RAG BIASA
rerata EM: 0.6300884955752213
rerata F1: 0.31540144816316523


In [ ]:
import json

# Simpan hasil jawaban berdasarkan T5 summary (selective)
with open("outputs/test_generated_answers/selective_generated_answers.json", "w", encoding="utf-8") as f:
    json.dump(answer_based_on_t5_summary, f, indent=2, ensure_ascii=False)

# Simpan hasil jawaban berdasarkan unselective summary
with open("outputs/test_generated_answers/unselective_generated_answers.json", "w", encoding="utf-8") as f:
    json.dump(answer_based_on_unselective_summary, f, indent=2, ensure_ascii=False)

# Simpan hasil jawaban berdasarkan original passages (normal RAG)
with open("outputs/test_generated_answers/normal_rag_generated_answer.json", "w", encoding="utf-8") as f:
    json.dump(answer_based_on_passages, f, indent=2, ensure_ascii=False)


## Dataset usang: 
Pembahasan di bawah adalah hasil eksekusi model pada dataset yang mana query bisa jadi tidak masuk ke prompt. Query bisa jadi tidak masuk kek prompt karena adanya truncation untuk maksimal 512 token, yang mana passages dan instruksi saja bisa mencapai batas tersebut  bahkan sebelum query diselipkan ke dalam prompt. Oleh karena itu, pada dataset terbaru, passages di-truncate thd ukuran `512 - jumlah token dari (instruksi+query)`

Di bawah, aku akan melakukan pengujian terhadap beberapa kasus berbeda:
1. Model yg dilatih thd dataset yg telah di-seed dan dataset latihan tsb ada seleksinya `(khalidrizki/T5base-RECOMP-seededDataset-withSelection)`
2. Model yg dilatih thd dataset yg telah di-seed dan dataset latihan tsb tiada seleksi
3. Model yg dilatih thd dataset yg belum di-seed dan dataset latihan tsb ada seleksi
4. Model yg dilatih thd dataset yg belum di-seed dan dataset latihan tsb tiada seleksi `(khalidrizki/T5base-RECOMP-unseedDataset-noSelection)`

Perangkuman oleh kompresor sudah dijalankan dengan command `python train_summarizer.py --model_name_or_path khalidrizki/{TERGANTUNG PAKAI MODEL APA} --do_predict --dataset_name khalidrizki/RECOMP-tuning --max_target_length 52 --output_dir ./outputs/ --per_device_eval_batch_size=32 --predict_with_generate --summary_column final_summary --seed 42`

Seharusnya: antar model yg sama2 melakukan with/no seleksi (tetapi hanya dibedakan oleh seed dataset saja) akan memiliki skor yang mirip. 

__Ukuran dataset kurang?__: antar model yg sama-sama un/seeded (tetapi berbeda pada diterapkan atau tidaknya seleksi), model yg dgn seleksi seharusnya lebih baik. Kalau tidak lebih baik, berarti ukuran dataset kurang. 

Idealnya

### T5base-RECOMP-seededDataset-withSelection

Sebelum menjalankan generasi jawaban dengan rangkuman hasil fine tuning T5, T5 sudah melakukan generasi rangkuman ketika menjalankan train_summarizer.py dengan command `python train_summarizer.py --do_predict`

In [3]:
# Muat baris-baris teks dari file
with open("./outputs/-khalidrizki-T5base-RECOMP-seededDataset-withSelection_on-recompTuning_hub_dataset/generated_predictions.txt", "r", encoding="utf-8", errors='replace') as f:
    predictions = [line.strip() for line in f.readlines()]

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions) == len(test_data), f"Jumlah prediksi ({len(predictions)}) tidak sama dengan jumlah data ({len(test_data)})"

AssertionError: Jumlah prediksi (564) tidak sama dengan jumlah data (565)

In [ ]:
while len(predictions) < len(test_data):
    print('jumlah prediksi kurang, meng-append string kosong ("")')
    predictions.append("")

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions) == len(test_data), f"Jumlah prediksi ({len(predictions)}) tidak sama dengan jumlah data ({len(test_data)})"

# Tambahkan kolom ke dataset
test_data = test_data.add_column("T5_summary", predictions)

from generate_answer import generate_answer_and_do_scoring

answer_based_on_t5_summary = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    filtered_contexts_col="T5_summary", 
    label_col='answer', 
    original_passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52
)

jumlah prediksi kurang, meng-append string kosong ("")


Generating responses with summary only:   0%|          | 0/565 [00:00<?, ?it/s]c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset

In [5]:
from datasets import Dataset
result = Dataset.from_list(answer_based_on_t5_summary)
em_avg = sum(result['em'])/len(result)
f1_avg =  sum(result['f1'])/len(result)
print("EM:" ,em_avg)
print("F1:", f1_avg)

EM: 0.10973451327433628
F1: 0.08678890872044633


In [ ]:

em_avg

0.11504424778761062

In [ ]:
f1_avg

0.0948697836281932

### T5base-RECOMP-unseedDataset-noSelection

In [ ]:
# Muat baris-baris teks dari file
with open("./outputs/-khalidrizki-T5base-RECOMP-unseedDataset-noSelection_on-recompTuning_hub_dataset/generated_predictions.txt", "r", encoding="utf-8", errors='replace') as f:
    predictions_unselective = [line.strip() for line in f.readlines()]

# Pastikan jumlah prediksi sama dengan jumlah data
assert len(predictions_unselective) == len(test_data), f"Jumlah prediksi ({len(predictions_unselective)}) tidak sama dengan jumlah data ({len(test_data)})"

# Tambahkan kolom ke dataset
test_data = test_data.add_column("unselective_summary", predictions_unselective)

from generate_answer import generate_answer_and_do_scoring

answer_based_on_unselective_summary = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    filtered_contexts_col="unselective_summary", 
    label_col='answer', 
    original_passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52
)

from datasets import Dataset
result_uns = Dataset.from_list(answer_based_on_unselective_summary)
em_uns_avg = sum(result_uns['em'])/len(result_uns)
f1_uns_avg =  sum(result_uns['f1'])/len(result_uns)
print("EM:" ,em_uns_avg)
print("F1:", f1_uns_avg)

Generating responses with summary only:   0%|          | 0/565 [00:00<?, ?it/s]c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset

EM: 0.4247787610619469
F1: 0.2505083083585362


### RAG konvensional

In [ ]:
from generate_answer import generate_answer_and_do_scoring
answer_based_on_passages = generate_answer_and_do_scoring(
    test_data, 
    query_col='query', 
    filtered_contexts_col="passages", 
    label_col='answer', 
    original_passages_col='passages', 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=52, 
    max_source_length = 512
)

from datasets import Dataset
normal_RAG = Dataset.from_list(answer_based_on_passages)
print("HASIL RAG BIASA")
print("rerata EM:", sum(normal_RAG['em'])/len(normal_RAG))
print("rerata F1:", sum(normal_RAG['f1'])/len(normal_RAG))

Generating responses with summary only: 100%|██████████| 565/565 [35:01<00:00,  3.72s/it]

HASIL RAG BIASA
rerata EM: 0.32035398230088497
rerata F1: 0.1765652985106173
